# Multi-Agent NuSMV Directory Checker (LangChain + OpenRouter) — With Reflection

Single file notebook implementing **single-run** multi-agent workload sharing plus **reflection**:

- Coordinator lists `.smv` files, partitions across workers, aggregates results.
- Worker agents call a custom tool that runs local **NuSMV** on macOS.
- After each worker returns JSON summaries, a **reflection pass** validates:
  - JSON schema
  - all assigned files covered
  - counts consistent with parsed specs
- If validation fails, a **reflection LLM** is invoked to repair the JSON (no tools), then it is re-validated.
- LLM: `mistralai/devstral-2512:free` via OpenRouter.

## Environment variables

Required:
- `OPENROUTER_API_KEY`

Recommended:
- `AGENT_ALLOWED_ROOT` (directory allowlist root)
- `SMV_DIR` (directory to analyze)
- `NUSMV_BIN` (path to NuSMV if not on PATH)
- `NUSMV_TIMEOUT_S` (timeout per file)
- `NUM_WORKERS` (worker count)


In [1]:
#%pip install -U langchain langchain-core langchain-openai


In [2]:
import os
os.environ["AGENT_ALLOWED_ROOT"]="./"
# Optional: NuSMV location if not on PATH
os.environ["NUSMV_BIN"]="./NuSMV"
# Optional: timeout per NuSMV run
os.environ["NUSMV_TIMEOUT_S"]="60"

In [ ]:
os.environ["OPENROUTER_API_KEY"] = "<API_KEY_HERE>"

In [4]:
import os
from pathlib import Path

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY env var.")

ALLOWED_ROOT = Path(os.getenv("AGENT_ALLOWED_ROOT", str(Path.cwd()))).resolve()
SMV_DIR = Path(os.getenv("SMV_DIR", str(Path.cwd()))).resolve()

NUSMV_BIN = os.getenv("NUSMV_BIN", "NuSMV")
DEFAULT_TIMEOUT_S = int(os.getenv("NUSMV_TIMEOUT_S", "60"))
NUM_WORKERS = int(os.getenv("NUM_WORKERS", "2"))

print("ALLOWED_ROOT:", ALLOWED_ROOT)
print("SMV_DIR:", SMV_DIR)
print("NUSMV_BIN:", NUSMV_BIN)
print("DEFAULT_TIMEOUT_S:", DEFAULT_TIMEOUT_S)
print("NUM_WORKERS:", NUM_WORKERS)


ALLOWED_ROOT: /Users/omersubasi/Downloads/AgentforModelChecking
SMV_DIR: /Users/omersubasi/Downloads/AgentforModelChecking
NUSMV_BIN: ./NuSMV
DEFAULT_TIMEOUT_S: 60
NUM_WORKERS: 2


In [5]:
import json
import math
import re
import shlex
import shutil
import subprocess
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

MAX_CHARS = 30000

def _resolve_under_allowed_root(p: Path) -> Path:
    rp = p.resolve()
    try:
        rp.relative_to(ALLOWED_ROOT)
    except ValueError as e:
        raise ValueError(f"Path must be under allowed root: {ALLOWED_ROOT}") from e
    return rp

def _find_nusmv_executable() -> str:
    candidate = Path(NUSMV_BIN)
    if candidate.is_absolute():
        if not candidate.exists():
            raise FileNotFoundError(f"NuSMV not found at: {candidate}")
        if not os.access(candidate, os.X_OK):
            raise PermissionError(f"NuSMV not executable: {candidate}")
        return str(candidate)

    resolved = shutil.which(NUSMV_BIN)
    if not resolved:
        raise FileNotFoundError(
            "NuSMV not found on PATH. Add it to PATH or set NUSMV_BIN to its absolute path."
        )
    return resolved

# --- Parsing (lightweight) ---
_SPEC_LINE_RE = re.compile(r"^\s*--\s*specification\s+(.*?)\s+is\s+(true|false)\s*$", re.IGNORECASE)
_COUNTEREX_RE = re.compile(
    r"^\s*--\s*as\s+demonstrated\s+by\s+the\s+following\s+execution\s+sequence",
    re.IGNORECASE,
)

@dataclass
class SpecResult:
    spec: str
    is_true: bool

def parse_nusmv_output(stdout: str, stderr: str) -> Dict[str, Any]:
    specs: List[SpecResult] = []
    counterexample_present = False

    for line in stdout.splitlines():
        m = _SPEC_LINE_RE.match(line)
        if m:
            spec_text = m.group(1).strip()
            truth = m.group(2).lower() == "true"
            specs.append(SpecResult(spec=spec_text, is_true=truth))
        if _COUNTEREX_RE.match(line):
            counterexample_present = True

    out_lines = stdout.splitlines()
    head = "\n".join(out_lines[:30])
    tail = "\n".join(out_lines[-30:]) if len(out_lines) > 30 else ""
    excerpt = head + ("\n...\n" + tail if tail else "")

    return {
        "specs": [{"spec": s.spec, "is_true": s.is_true} for s in specs],
        "counterexample_present": counterexample_present,
        "stderr_nonempty": bool(stderr.strip()),
        "excerpt": excerpt[:MAX_CHARS],
    }

# --- Fix A: Pure functions ---
def list_smv_files_fn(directory: str) -> Dict[str, Any]:
    d = _resolve_under_allowed_root(Path(directory))
    if not d.exists():
        raise FileNotFoundError(f"Directory not found: {d}")
    if not d.is_dir():
        raise ValueError(f"Not a directory: {d}")

    files = sorted([p.resolve() for p in d.iterdir() if p.is_file() and p.suffix.lower() == ".smv"])
    safe_files = [str(_resolve_under_allowed_root(p)) for p in files]
    return {"directory": str(d), "files": safe_files, "count": len(safe_files)}

def run_nusmv_on_file_fn(smv_file: str, extra_args: Optional[str] = None, timeout_s: int = DEFAULT_TIMEOUT_S) -> Dict[str, Any]:
    exe = _find_nusmv_executable()
    f = _resolve_under_allowed_root(Path(smv_file))

    if not f.exists():
        raise FileNotFoundError(f"File not found: {f}")
    if not f.is_file():
        raise ValueError(f"Not a regular file: {f}")
    if f.suffix.lower() != ".smv":
        raise ValueError(f"Not an .smv file: {f}")

    cmd = [exe, str(f)]
    if extra_args:
        cmd.extend(shlex.split(extra_args))

    try:
        p = subprocess.run(cmd, text=True, capture_output=True, timeout=timeout_s, check=False)
        stdout = (p.stdout or "")[:MAX_CHARS]
        stderr = (p.stderr or "")[:MAX_CHARS]
        parsed = parse_nusmv_output(stdout=stdout, stderr=stderr)
        return {"file": str(f), "command": cmd, "exit_code": p.returncode, "stdout": stdout, "stderr": stderr, "parsed": parsed}
    except subprocess.TimeoutExpired as e:
        stdout = (e.stdout or "")[:MAX_CHARS]
        stderr = (e.stderr or "")[:MAX_CHARS]
        return {"file": str(f), "command": cmd, "exit_code": None, "stdout": stdout, "stderr": stderr, "error": f"Timed out after {timeout_s} seconds"}

def compute_directory_summary(results: List[Dict[str, Any]]) -> Dict[str, Any]:
    files_total = len(results)
    files_succeeded = 0
    files_failed_run = 0
    files_with_failed_specs = 0

    specs_total = 0
    specs_true = 0
    specs_false = 0

    counterexample_files: List[str] = []
    failed_specs_by_file: Dict[str, List[str]] = {}

    for r in results:
        file_path = r.get("file", "<unknown>")
        exit_code = r.get("exit_code", None)
        run_ok = (exit_code == 0) and ("error" not in r)

        if run_ok:
            files_succeeded += 1
        else:
            files_failed_run += 1

        parsed = (r.get("parsed") or {})
        specs = parsed.get("specs") or []
        specs_total += len(specs)

        false_specs = []
        for s in specs:
            if s.get("is_true") is True:
                specs_true += 1
            elif s.get("is_true") is False:
                specs_false += 1
                false_specs.append(s.get("spec", "<unknown spec>"))

        if false_specs:
            files_with_failed_specs += 1
            failed_specs_by_file[file_path] = false_specs

        if parsed.get("counterexample_present") is True:
            counterexample_files.append(file_path)

    pass_rate = (specs_true / specs_total) if specs_total else None

    return {
        "files_total": files_total,
        "files_succeeded": files_succeeded,
        "files_failed_run": files_failed_run,
        "files_with_failed_specs": files_with_failed_specs,
        "specs_total": specs_total,
        "specs_true": specs_true,
        "specs_false": specs_false,
        "spec_pass_rate": pass_rate,
        "counterexample_files": counterexample_files,
        "failed_specs_by_file": failed_specs_by_file,
    }


In [6]:
from langchain_core.tools import tool

@tool("run_nusmv_on_file")
def run_nusmv_on_file(smv_file: str, extra_args: Optional[str] = None, timeout_s: int = DEFAULT_TIMEOUT_S) -> Dict[str, Any]:
    """Run NuSMV on a single .smv file and return structured output."""
    return run_nusmv_on_file_fn(smv_file, extra_args=extra_args, timeout_s=timeout_s)


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents import create_tool_calling_agent

def build_openrouter_llm() -> ChatOpenAI:
    return ChatOpenAI(
        model="mistralai/devstral-2512:free",
        api_key=OPENROUTER_API_KEY,
        base_url="https://openrouter.ai/api/v1",
        temperature=0.2,
    )

# NOTE: literal braces are escaped as {{ and }} to avoid template-variable parsing.
WORKER_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a NuSMV verification worker.\n"
            "You will receive a list of .smv files to process.\n"
            "For each file, call run_nusmv_on_file.\n"
            "Return STRICT JSON only (no markdown, no commentary) with schema:\n"
            "{{\n"
            '  "worker_id": <int>,\n'
            '  "results": [\n'
            "    {{\n"
            '      "file": <str>,\n'
            '      "exit_code": <int|null>,\n'
            '      "error": <str|null>,\n'
            '      "parsed": {{\n'
            '        "specs": [{{"spec": <str>, "is_true": <bool>}}],\n'
            '        "counterexample_present": <bool>,\n'
            '        "stderr_nonempty": <bool>\n'
            "      }}\n"
            "    }}\n"
            "  ]\n"
            "}}\n"
            "Rules: Do not invent values; rely only on tool output."
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

def build_worker(worker_id: int) -> AgentExecutor:
    llm = build_openrouter_llm()
    tools = [run_nusmv_on_file]
    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=WORKER_PROMPT)
    return AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=12,
        tags=[f"worker-{worker_id}"],
    )

# Reflection LLM: repairs worker JSON if validator finds issues.
REFLECTION_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a strict JSON repair assistant.\n"
            "Given assigned_files, prior_json, and issues, output corrected STRICT JSON only.\n"
            "Do not add commentary, markdown, or extra keys beyond the schema.\n"
            "Schema:\n"
            "{{\n"
            '  "worker_id": <int>,\n'
            '  "results": [{{"file": <str>, "exit_code": <int|null>, "error": <str|null>, "parsed": {{"specs": [{{"spec": <str>, "is_true": <bool>}}], "counterexample_present": <bool>, "stderr_nonempty": <bool>}} }}] \n'
            "}}"
        ),
        ("human", "{input}"),
    ]
)


In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def chunk_list(items: List[str], n_chunks: int) -> List[List[str]]:
    if n_chunks <= 0:
        return [items]
    if not items:
        return [[] for _ in range(n_chunks)]
    chunk_size = math.ceil(len(items) / n_chunks)
    return [items[i:i + chunk_size] for i in range(0, len(items), chunk_size)]

def make_worker_input(worker_id: int, files: List[str], extra_args: Optional[str], timeout_s: int) -> str:
    lines = [
        f"worker_id={worker_id}",
        "Process the following SMV files (absolute paths):",
        *[f"- {fp}" for fp in files],
        "",
        f"Use timeout_s={timeout_s}.",
        f"extra_args={(extra_args if extra_args else 'null')}.",
        "Return STRICT JSON only.",
    ]
    return "\n".join(lines)

def _extract_json_object(text: str) -> Dict[str, Any]:
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        pass
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return json.loads(text[start:end+1])
    raise ValueError("Could not parse JSON from worker output.")

def validate_worker_json(obj: Dict[str, Any], assigned_files: List[str]) -> List[str]:
    issues: List[str] = []
    if not isinstance(obj, dict):
        return ["Output is not a JSON object."]

    if "worker_id" not in obj or not isinstance(obj["worker_id"], int):
        issues.append("Missing/invalid worker_id (must be int).")

    results = obj.get("results")
    if not isinstance(results, list):
        issues.append("Missing/invalid results (must be list).")
        return issues

    seen_files = []
    for i, r in enumerate(results):
        if not isinstance(r, dict):
            issues.append(f"results[{i}] is not an object.")
            continue
        f = r.get("file")
        if not isinstance(f, str):
            issues.append(f"results[{i}].file missing/invalid.")
        else:
            seen_files.append(f)

        # Basic parsed schema
        parsed = r.get("parsed")
        if not isinstance(parsed, dict):
            issues.append(f"results[{i}].parsed missing/invalid.")
            continue

        specs = parsed.get("specs")
        if not isinstance(specs, list):
            issues.append(f"results[{i}].parsed.specs missing/invalid (must be list).")
        else:
            for j, s in enumerate(specs):
                if not isinstance(s, dict) or "spec" not in s or "is_true" not in s:
                    issues.append(f"results[{i}].parsed.specs[{j}] missing spec/is_true.")

        if not isinstance(parsed.get("counterexample_present", False), bool):
            issues.append(f"results[{i}].parsed.counterexample_present must be bool.")
        if not isinstance(parsed.get("stderr_nonempty", False), bool):
            issues.append(f"results[{i}].parsed.stderr_nonempty must be bool.")

    # Coverage check
    missing = sorted(set(assigned_files) - set(seen_files))
    extra = sorted(set(seen_files) - set(assigned_files))
    if missing:
        issues.append(f"Missing results for assigned files: {missing}")
    if extra:
        issues.append(f"Includes files not assigned to this worker: {extra}")

    return issues

def repair_with_reflection_llm(worker_id: int, assigned_files: List[str], prior_obj: Any, issues: List[str]) -> Dict[str, Any]:
    llm = build_openrouter_llm()
    chain = REFLECTION_PROMPT | llm
    payload = {
        "assigned_files": assigned_files,
        "issues": issues,
        "prior_json": prior_obj,
        "required_worker_id": worker_id,
    }
    out = chain.invoke({"input": json.dumps(payload, indent=2)}).content
    obj = _extract_json_object(out)
    return obj

def run_worker_with_reflection(worker_id: int, worker: AgentExecutor, batch: List[str], extra_args: Optional[str], timeout_s: int) -> Dict[str, Any]:
    inp = make_worker_input(worker_id, batch, extra_args=extra_args, timeout_s=timeout_s)
    out = worker.invoke({"input": inp})["output"]
    obj = _extract_json_object(out)
    issues = validate_worker_json(obj, batch)
    if not issues:
        return obj

    # One repair attempt
    repaired = repair_with_reflection_llm(worker_id, batch, obj, issues)
    repaired_issues = validate_worker_json(repaired, batch)
    if repaired_issues:
        # Return best-effort with validation notes
        return {
            "worker_id": worker_id,
            "results": obj.get("results", []) if isinstance(obj, dict) else [],
            "validation_failed": True,
            "issues": issues,
            "repaired_issues": repaired_issues,
        }
    return repaired

def coordinator_run_directory_single_run_reflect(directory: Path, num_workers: int = 4, extra_args: Optional[str] = None, timeout_s: int = DEFAULT_TIMEOUT_S) -> Dict[str, Any]:
    listing = list_smv_files_fn(str(directory))
    files = listing["files"]

    if not files:
        return {
            "directory": listing["directory"],
            "files": [],
            "results": [],
            "overall_summary": compute_directory_summary([]),
            "worker_json": [],
            "note": "No .smv files found.",
        }

    batches = chunk_list(files, num_workers)
    batches = [b for b in batches if b]
    workers = [build_worker(i) for i in range(len(batches))]

    worker_json: List[Dict[str, Any]] = []
    with ThreadPoolExecutor(max_workers=len(batches)) as pool:
        futures = {}
        for worker_id, (worker, batch) in enumerate(zip(workers, batches)):
            fut = pool.submit(run_worker_with_reflection, worker_id, worker, batch, extra_args, timeout_s)
            futures[fut] = worker_id

        for fut in as_completed(futures):
            worker_json.append(fut.result())

    # Aggregate results from workers (single-run; no re-execution)
    all_results: List[Dict[str, Any]] = []
    for obj in sorted(worker_json, key=lambda x: x.get("worker_id", 0)):
        rs = obj.get("results", [])
        if isinstance(rs, list):
            all_results.extend(rs)

    normalized: List[Dict[str, Any]] = []
    for r in all_results:
        if not isinstance(r, dict):
            continue
        normalized.append(
            {
                "file": r.get("file"),
                "exit_code": r.get("exit_code"),
                "error": r.get("error"),
                "parsed": r.get("parsed") or {},
            }
        )

    overall_summary = compute_directory_summary(normalized)

    return {
        "directory": listing["directory"],
        "files": files,
        "results": normalized,
        "overall_summary": overall_summary,
        "worker_json": worker_json,
    }


In [9]:
REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a verification lead.\n"
            "Write a concise report using ONLY the provided structured results.\n"
            "Include:\n"
            "1) Overall directory summary\n"
            "2) Per-file summary (call out run failures/timeouts)\n"
            "3) Key risks and recommended next actions\n"
            "Do not fabricate tool outputs."
        ),
        ("human", "{input}"),
    ]
)

def summarize_directory_run(run: Dict[str, Any]) -> str:
    llm = build_openrouter_llm()
    chain = REPORT_PROMPT | llm

    payload = {
        "directory": run["directory"],
        "overall_summary": run["overall_summary"],
        "per_file": [
            {
                "file": r.get("file"),
                "exit_code": r.get("exit_code"),
                "error": r.get("error"),
                "specs_total": len((r.get("parsed") or {}).get("specs") or []),
                "specs_false": sum(1 for s in ((r.get("parsed") or {}).get("specs") or []) if s.get("is_true") is False),
                "counterexample_present": (r.get("parsed") or {}).get("counterexample_present", False),
                "stderr_nonempty": (r.get("parsed") or {}).get("stderr_nonempty", False),
            }
            for r in run["results"]
        ],
        "worker_validation_notes": [
            {k: v for k, v in w.items() if k in ["worker_id", "validation_failed", "issues", "repaired_issues"]}
            for w in run.get("worker_json", [])
            if isinstance(w, dict) and w.get("validation_failed") is True
        ],
    }

    return chain.invoke({"input": json.dumps(payload, indent=2)}).content


In [10]:
# Execute multi-agent run (single NuSMV run per file, via workers) with reflection/repair
run = coordinator_run_directory_single_run_reflect(
    directory=SMV_DIR,
    num_workers=NUM_WORKERS,
    extra_args=None,            # e.g. "-dynamic"
    timeout_s=DEFAULT_TIMEOUT_S,
)

print("Structured overall summary:")
print(json.dumps(run["overall_summary"], indent=2))

if run.get("worker_json"):
    print("\nWorker reflection status:")
    for w in sorted(run["worker_json"], key=lambda x: x.get("worker_id", 0)):
        flag = w.get("validation_failed", False)
        print(f"- worker {w.get('worker_id')}: validation_failed={flag}")




> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...


Invoking: `run_nusmv_on_file` with `{'smv_file': '/Users/omersubasi/Downloads/AgentforModelChecking/short.smv', 'timeout_s': 60}`


{'file': '/Users/omersubasi/Downloads/AgentforModelChecking/short.smv', 'command': ['./NuSMV', '/Users/omersubasi/Downloads/AgentforModelChecking/short.smv'], 'exit_code': 0, 'stdout': '*** This is NuSMV 2.7.0 (compiled on Thu Oct 24 15:29:16 2024)\n*** Enabled addons are: compass\n*** For more information on NuSMV see <http://nusmv.fbk.eu>\n*** or email to <nusmv-users@list.fbk.eu>.\n*** Please report bugs to <Please report bugs to <nusmv-users@fbk.eu>>\n\n*** Copyright (c) 2010-2024, Fondazione Bruno Kessler\n\n*** This version of NuSMV is linked to the CUDD library version 2.4.1\n*** Copyright (c) 1995-2004, Regents of the University of Colorado\n\n*** This version of NuSMV is linked to the MiniSat SAT solver. \n*** See http://minisat.se/MiniSat.html\n*** Copyright (c) 2003

In [11]:
# Generate a human-readable report
report = summarize_directory_run(run)
print(report)


**Verification Report**

### **Overall Directory Summary**
- **Files:** 3 total, 0 succeeded, 3 failed (run failures/timeouts).
- **Specifications:** 5 total, 3 true, 2 false (60% pass rate).
- **Counterexamples found in:**
  - `mutex.smv`
  - `semaphore.smv`

### **Per-File Summary**
1. **`mutex.smv`**
   - **Status:** Run failed (exit code 0, no error).
   - **Specs:** 3 total, 1 false.
   - **Counterexample:** Present.
   - **Failed Spec:** `EF (state1 = c1 & state2 = c2)`.

2. **`semaphore.smv`**
   - **Status:** Run failed (exit code 0, no error).
   - **Specs:** 1 total, 1 false.
   - **Counterexample:** Present.
   - **Failed Spec:** `AG (proc1.state = entering -> AF proc1.state = critical)`.
   - **Note:** Non-empty stderr (possible warnings/errors).

3. **`short.smv`**
   - **Status:** Run failed (exit code 0, no error).
   - **Specs:** 1 total, 0 false.
   - **Counterexample:** None.

### **Key Risks & Recommended Next Actions**
- **Risks:**
  - Two files (`mutex.smv`, `semap